In [ ]:
from window_det import window_det
from omni_seg_centered import omni_seg
import csv
import pandas as pd
import numpy as np
import glob
from gipm_transform import gipm_transform
from cone_angle_dfs_dt_ind import cone_angle_dfs
from gipm_plot_a import gipm_plot_a


In [ ]:
#read in raw CSVs for Cluster

cluster_csv_list = []

path = "/Users/apx059/Documents/Cluster CSVs 2001-02-01-2001-03-15/**"
list_all = []

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders

for element in list_all:
    if '.csv' in element:
        cluster_csv_list.append(element)
        
cluster_dfs = []

for file in cluster_csv_list:
    df = pd.read_csv(file)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
    cluster_dfs.append(df)

print(cluster_dfs[0])

In [ ]:
#append OMNI dfs to each other

omni_csv_list = [r"/Users/apx059/Documents/OMNI CSVs New/om_2001_Feb.csv",
r"/Users/apx059/Documents/OMNI CSVs New/om_2001_Mar_01_15.csv"]
omni_df_list = []

for file in omni_csv_list:
    df = pd.read_csv(file)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
    omni_df_list.append(df)

omni_df = pd.concat(omni_df_list)



In [ ]:
f_winds_all = []

for df in cluster_dfs:
    f_winds = window_det(df)
    f_winds_all.append(f_winds)
    
    

In [ ]:
%%time

om_ave_dfs_list = []

for wind_list in f_winds_all:
    om_ave = omni_seg(omni_df, wind_list)
    om_ave['datetime']=pd.to_datetime(om_ave['datetime'])
    om_ave = om_ave.set_index('datetime')
    om_ave_dfs_list.append(om_ave)

print(om_ave_dfs_list[0])

In [ ]:
#find GIPM rotation matrices and scaling coefficient for every two minute interval
GIPM_mat_list = []
FAC_list = []

for df in om_ave_dfs_list:
    GIPM_mat, FAC_coeffs = gipm_transform(df)
    GIPM_mat_list.append(GIPM_mat)
    FAC_list.append(FAC_coeffs)
    

In [ ]:
%%time
#use GIPM matrices and scaling coeff to generate average GIPM location for spacecraft in 
#each two minute interval.

Cluster_GIPM_loc_all = []

for f_w,cl_df,GIPM_mat,FAC_c in zip(f_winds_all, cluster_dfs, GIPM_mat_list, FAC_list):
    Cluster_GIPM_loc = gipm_plot_a(f_w, cl_df, GIPM_mat, FAC_c)
    Cluster_GIPM_loc_all.append(Cluster_GIPM_loc)
    

In [ ]:
%%time
#split by cone angle
rad_df_list = []
spir_df_list = []
perp_df_list = []

for om_ave, cl_df, GIPM_lc in zip(om_ave_dfs_list, cluster_dfs, Cluster_GIPM_loc_all):
    rad_df, spir_df, perp_df = cone_angle_dfs(om_ave,cl_df, GIPM_lc)
    rad_df_list.append(rad_df)
    spir_df_list.append(spir_df)
    perp_df_list.append(perp_df)


In [ ]:
#save files as CSVs
#omni first

for df in om_ave_dfs_list:
    firstwin = df.index[0]
    firstwin = str(firstwin)
    fpath = '/Users/apx059/Documents/OMNI Processed CSVs New/OMNI_averages_' + firstwin + '.csv'
    df.to_csv(fpath)
    

In [ ]:
#save files as CSVs
#now cluster

for df in rad_df_list:
    if df.size > 0:
        firstwin = df.loc[0, 'window start']
        firstwin = str(firstwin)
        fname = '/Users/apx059/Documents/Radial Cluster GIPM Locs/rad_Cluster_loc_' + firstwin + '.csv'
        df.to_csv(fname)
    
for df in spir_df_list:
    if df.size > 0:
        firstwin = df.loc[0, 'window start']
        firstwin = str(firstwin)
        fname = '/Users/apx059/Documents/Spiral Cluster GIPM Locs/spir_Cluster_loc_' + firstwin + '.csv'
        df.to_csv(fname)
    
for df in perp_df_list:
    if df.size > 0:
        firstwin = df.loc[0, 'window start']
        firstwin = str(firstwin)
        fname = '/Users/apx059/Documents/Perp Cluster GIPM Locs/perp_Cluster_loc_' + firstwin + '.csv'
        df.to_csv(fname)
    